# Neuronale Netze mit Python und TensorFlow

Im Gegensatz zum vorherigen Artikel über TensorFlow, handelt sich hier eher um ein Tutorial, wie man TensorFlow verwendet um ein neuronales Netz zu erstellen und zu trainieren. Der Artikel geht davon aus, dass man über Python-Kentnisse verfügt und TensorFlow und Scikit-Learn bereits installiert hat. [(Wie installiert man TensorFlow)](https://www.tensorflow.org/install/)

Wir fangen an mit einem Vergleich zwischen echten und künstlichen neuronalen Netzen:

## Echte Neuronale Netzwerke

![nervenzelle](images/neuron.png)

*Nervenzelle; Autor: [Quasar Jarosz](https://en.wikipedia.org/wiki/User:Quasar_Jarosz); Licenz: [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0)*

Gezeigt ist ein Neuron. Im menschlichen Gehirn gibt es etwa 86 Milliarden davon. Die wichtigsten Strukturen sind:

- **Dendrite** - erhalten Signale von anderen Nervenzellen, wobei ein Neuron kann sehr viele Dendrite haben
- **Zellkörper** - summiert die Signale um die Ausgabe zu generieren
- **Axon** - wenn die Summe einen Schwellwert erreicht, wird ein Signal über den Axon übertragen. Nervenzelle haben immer nur einen Axon
- **Axonterminale (Synapse)** - Der Verbindungspunkt zwischen Axon und Dendriten. Die Stärke der Verbindung entspricht der Stärke des übertragten Signal (synaptische Gewichte)

## Künstliche Neuronale Netz

Perzeptronen und Sigmoidneurone sind die Hauptbestandteile eines neuronalen Netzes.

![Perzeptrone](images/perceptron.png)

*Bild von Perzeptronen. Ein mit, der andere ohne Bias*

Ähnlich wie Neuronen, haben Perzeptronen mehrere **Inputs** (*x*) mit entsprechenden **Gewichtungen** (*w*). Die Outputs sind entweder 0 oder 1. Jeder Input wird mit einer entsprechenden Gewichtung multipliziert. Am Ende summiert man alle Ergebnisse und addiert noch die **Bias** (*b oder Theta*), welche dem **Schwellwert** entspricht. Angenommen, man hat *N* Inputs, sieht die Formel folgendermaßen aus:

![Perceptron Equation](images/perceptron_equation.png)

Ist die Gesamtsumme größer 0 ist der Output 1, ist die Summe kleiner 0 ist der Output entsprechend 0.

Perzeptronen sind oft zu primitiv, weswegen Sigmoidneuronen verwendet werden. Der Unterschied ist, dass sie auch eine **Aktivierungsfunktion** haben. 

![Sigmoidfunktion](images/sigmoid_fn.png)

Diese Funktion erlaubt, dass der Output der Sigmoidneuronen auch Zahlen zwischen 0 und 1 beinhaltet, statt nur 0 oder 1 wie im Perzeptron. Der große Vorteil ist, dass somit kleine Anpassungen an den Gewichtungen und Biasen nur in kleinen Unterschieden in der Ausgabe resultieren.

![neuronales Netz](images/neural_network.png)

*Neuronales Netz von [Glosser.ca](https://commons.wikimedia.org/wiki/User_talk:Glosser.ca), lizensiert unter [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0)*

Ein neuronales Netz besteht aus mehrere Schichten, wobei jede Schicht aus mehreren Neuronen besteht. In der Regel hat jedes Neuron aus einer Schicht Verbindungen zu allen anderen Neuronen aus der nächsten Schicht. Die erste Schicht ist die Inputschicht, wo der Datensatz eingeht. Dahinter können beliebig viele versteckte Schichten liegen, aber die letzte ist die Outputschicht, mit 1 oder mehreren Neuronen (Die Anzahl hängt vom Problem, den man lösen möchte ab).

## Funktionsweise von TensorFlow

Die Hauptdatenstruktur in TensorFlow ist ein Tensor. Hierbei handelt es sich um ein Array von primitiven Datentypen, die in eine beliebige Anzahl von Dimensionen geformt sind. Der Rang eines Tensoren ist die Anzahl seiner Dimensionen.

```python
# Tensor Beispiele

3 # Rang 0
[1, 2, 3] # Rang 1
[[1,2,3], [1,2,3]] # Rang 2
[[[1,2,3], [1,2,3]]] # Rang 3
```

Einfache Arten von Tensoren sind Konstanten und Unbekannten (*Constant, Variable*). Konstanten sind, wie der Name schon sagt, Tensoren, deren Wert nicht verändert werden kann. Während der Ausführung des gesamten Programms behalten sie den gleichen Wert. Andererseits können die Werte von Variablen geändert werden, weshalb sie als Gewichte und Bias für die verschiedenen Schichten der Modelle verwendet werden.

Es ist auch wichtig, die Tensoren zu erwähnen, die das Ergebnis einer Operation mit anderen Tensoren enthalten. Beispiele sind Addition, Multiplikation oder die Anwendung einer komplexeren Funktion.

![dataflow graph](images/tensors_flowing.gif)

All dies bildet eine Art Graph, die aus allen Inputs, Outputs und Operationen dazwischen besteht. Der Graph ist nur eine Beschreibung der Datentypen der Eingaben und der Reihenfolge der Operationen. Um den Graph "auszuführen" muss man ein **Session**-Objekt verwenden. Dieser Ansatz bietet die Möglichkeit, das Modell auf separat und parallel auf mehreren CPUs und GPUs laufen zu lassen, was zu schnellere Trainingszeiten führt. Außerdem hat man die Möglichkeit teile der Grafen für andere Modelle wiederzuverwenden, oder den ganzen Graph mittels **TensorBoard** zu visualisieren.

In [1]:
import tensorflow as tf

a = tf.constant(2.0)
b = tf.constant(3.0)
c = a + b

# Das zeigt uns nur, dass dieser Tensor die Operation Addition asuführt
print(c)

Tensor("add:0", shape=(), dtype=float32)


In [2]:
# mit einer Session können wir die Berechnung ausführen und das Ergebnis ausgeben
sess = tf.Session()
sess.run(c)

5.0

## Implementierung eines neuronalen Netzes

Nun werden wir ein einfaches neuronales Netz mit TensorFlow implementieren. Zusätzlich brauchen wir aber einige Funktionen aus *scikit-learn*. Wir werden den [Iris-Datensatz](https://en.wikipedia.org/wiki/Iris_flower_data_set) benutzen und versuchen mit dem neuronalen Netz die Blumen richtig zu klassifizieren.

Als erstes wollen wir Tensorflow und einige Funktionen aus *scikit-learn* importieren. Dann laden wir den Irisdatensatz.

In [3]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Datensatz
data = load_iris()
features = data.data
labels = data.target.reshape((-1, 1))

# Klassen von Blumen zeigen
labels

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

### One-Hot Encoding

Wie man hier sieht, sind die Blummenklassen mit 0, 1 oder 2 bezeichnet. In der Regel, wenn man neuronale Netze für Klassifizierungsprobleme benutzen will, hat man so viele Neuronen in der letzten Schicht wie Klassen. Wir erwarten für jede Stichprobe eine Ausgabe, die so Aussieht `[0.15, 0.70, 0.15]`. In diesem Beispiel ist die Klasse der Blume 1, weil die zweite Zahl die größte ist.

Wir wollen aber die Vorhersagen des neuronalen Netzes mit den echten Klassen vergleichen, deswegen wandeln wir die numerische Darstellung der Klassen zur sogenannten *One-hot encoding*. So hat man eine 3-Array als Bezeichner für jede Klasse.

Beispiel:
```
Klasse 0 -> [1, 0, 0]
Klasse 1 -> [0, 1, 0]
Klasse 2 -> [0, 0, 1]
```

Wir verwenden dafür die `OneHotEncoder` von *scikit-learn*

In [4]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
enc.fit_transform(labels)

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.

### Trainings- und Testdaten

Um das Modell gut validieren zu können, sollen wir Daten verwenden, mit denen nie trainiert worden ist. Deswegen benutzen wir die `train_test_split` Funktion, um 20 Prozent der Daten als Testdaten zu nehmen. Dann merken wir uns die Anzahl von Merkmale bzw. von Klassen. In diesem Fall ist `x_size = 4` und `y_size = 3`

In [5]:
# Trainings- und Testdaten erzeugen
train_x, test_x, train_y, test_y = train_test_split(
                                features, 
                                enc.transform(labels) )

x_size = train_x.shape[1]
y_size = train_y.shape[1]

**Placeholder** (Platzhalter) sind Tensors, die für die Dateneingabe sorgen. Man muss nur die Dimensionen spezifizieren. Zum Beispiel sind hier die Dimensionen für *X* `[None, x_size]`, weil wir eine unbestimmte Zahl von Stichproben haben mit jeweils `x_size` Merkmalen.

In *X* geben wir die Merkmale ein, und in *Y* geben wir die korrekte Klassen ein, damit wir das Modell trainieren können.

In [6]:
X = tf.placeholder(tf.float32, shape=[None, x_size])
Y = tf.placeholder(tf.float32, shape=[None, y_size])


Für das Modell des neuronales Netzes benutzen wir 2 Zwischenschichten, mit jeweils 128 Neuronen. Wir benutzen verschiedene Aktivierungsfunktionen für die Zwischen- und Ausgabeschichten. Für die Ausgabeschicht benutzen wir die `softmax` Aktivierungsfunktion. Diese erlaubt uns eine Wahrscheinlichkeit für die Ausgabeklassen zu definieren, da die Summe aller Elemente aus der Ausgabeliste wegen der Aktivierungsfunktion gleich `1` ist. Beispiel für Ausgabe: `[0.10, 0.78. 0.12]`. Das heißt, dass das Modell 78% sicher ist, dass die aktuelle Stichprobe der Klasse `1` ist.

Die `softmax` Funktion funktioniert nicht so gut für die Zwischenschichten, deswegen haben wir die `sigmoid` Aktivierungsfunktion ausgewählt.

TensorFlow bietet viele Werkzeuge an, mit denen man ein Modell erstellen kann. Wir schauen uns hier zwei Möglichkeiten. Zum ersten definieren wir die Schichten mit TensorFlow Core. Man braucht mehr Codezeilen, dafür aber hat man mehr Kontrolle über das Program. Die zweite Lösung verwendet die API `tf.layers`, womit man schnell neue Schichten definieren kann.

#### TensorFlow Core
Um schneller das Modell erstellen zu können, definieren wir eine eigene Funktion `hidden_layer`, mit den folgenden Parametern.
- `input` - Die Eingabetensor oder auch die vorige Schicht
- `shape` - Eine 2-Array mit den Dimensionen der Schicht. Das erste Element ist die Neuronenanzahl der vorigen Schicht und das zweite - die Neuronenanzahl dieser Schicht.
- `activation` - Aktivierungsfunktion. Da wir für verschiedene Schichten, unterschiedliche Aktivierungsfunktionen verwenden wollen, brauchen wir diese als Parameter einzugeben.

Mit `random_normal(shape)` generieren wir zufällige Werte mit bestimmten Dimensionen für Gewichtungen und Biases. Danach bilden wir Unbekannten, deren Werte man mit einem Optimierer anpassen kann. 

Die algebraische Operationen, die im neuronalen Netz stattfinden, sind folgenderweise definiert:
- man multipliziert das Input mit den Gewichtungen
- man addiert die Biases dazu
- man wendet die Aktivierungsfunktion an das Ergebnis an

Man bemerkt, dass die Dimensionen der Gewichtungen und der Biases in den versteckten Schichten sich unterschieden. In der ersten Schicht hat das Input Dimensionen `1x4`, weil jede Stichprobe 4 Merkmale hat. Die Gewichtungen haben die Dimension `4x128`, weil wir 128 Neuronen haben und in jedem sollen wir eine Gewichtung pro Merkmal haben. Nachdem wir `tf.matmul()` ausführen, kommt ein Ergebnis heraus mit den Dimensionen 1x128. Um die Biases dazu zu addieren, brauchen die dieselbe Dimensionen zu haben. Bei einer Stichprobe **x**, Gewichtungen für jedes Neuron **W**, und Bias **b**, sieht die Formel für die erste versteckte Schicht folgendermaßen aus:

![neural network equation](images/nn_equation.png)

#### `tf.layers`

Mit `tf.layers` ist es sehr einfach ein neuronales Netz zu definieren. `tf.layers.dense` bietet uns das typische Schichtenmodell und behandelt automatisch die Addition von Biases und die Berechnung von Dimensionen. Wir geben nur das Eingabetensor, die Neuronenanzahl und die Aktivierungsfunktion ein. 

Es gibt auch andere Parameter, die man anpassen kann, die aber für uns im Moment nicht relevant sind. Bei Interesse, kann man sich die Dokumentation anschauen.

In [9]:
# NN Modell mit TF Core
def hidden_layer(inputs, shape, activation=tf.nn.sigmoid):
    weights = tf.Variable(tf.random_normal(shape))
    biases = tf.Variable(tf.random_normal([1, shape[1]]))
    
    return activation(tf.add(tf.matmul(inputs, weights), biases))

core_layer1 = hidden_layer(X, [x_size, 128])
core_layer2 = hidden_layer(core_layer1, [128, 128])
core_y_hat = hidden_layer(core_layer1, [128, y_size], tf.nn.softmax)

In [11]:
# MM Modell mit tf.layers
# wir wollen die Variablen nicht überschreiben
h_layer1 = tf.layers.dense(X, 128, activation=tf.nn.sigmoid)
h_layer2 = tf.layers.dense(h_layer1, 128, activation=tf.nn.sigmoid)
y_hat = tf.layers.dense(h_layer2, y_size, activation=tf.nn.softmax)

### Kostenfunktion, Optimierer und Session

Wir brauchen jetzt eine **Kostenfunktion** um den Fehler zwischen den Vorhersgen und die echten Klassen zu berechnen. Die Tensorflow Funktion `softmax_cross_entropy_with_logits()` ist eine sehr gute Kostenfunktion, wenn man ein Klassifikationsproblem hat. Die Rückgabewert dieser Funktion ist eine Liste, mit den Fehlern der Vorhersage. Wir verwenden dann die Funktion `reduce_mean()` um den Mittelwert dieser Fehler zu berechnen, um nur mit einer Zahl zu arbeiten.

Danach brauchen wir einen Optimierer, damit wir die Gewichtungen in den vorigen Schichten anpassen zu können. Den `GradientDescentOptimizer` kann man hier gut anwenden und das Ziel ist die Kostenfunktion zu minimieren. Als argument gibt man die Lernrate ein, wobei man darauf Achten muss, dass eine zu große Zahl zu einem ungenaueren Modell führen kann, und eine zu kleine Zahl zu lange Trainingszeiten führen kann.

Bevor wir mit der Ausführung des Programms startet, müssen wir die Unbekannten initializieren. Diese sind die Gewichtungen und Biases in den Neuronenschichten, die der Optimierer anpasst, um die Kostenfunktion zu minimieren. Die Unbekannten haben am Anfang keine Werte und wir müssen den `global_variables_initializer()` ausführen um zufällige Werte da zu setzen.

In [13]:
# Diese Variable modifizieren um die zwei Modelle zu testen
# (y_hat ODER core_y_hat)
y_pred = core_y_hat

# Kosten- und Optimierungsfunktion
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=Y, logits=y_pred))

train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

# Unbekannten initialisieren und Session erstellen
sess = tf.Session()
sess.run(tf.global_variables_initializer())

### Trainingsschleife

Die erste Schleife ist für die Anzahl der Epochen. In diesem Fall wollen wir 300 mal alle Stichproben dem neuronalen Netz eingeben und zwar einer nach dem anderen (die zweite Schleife).

Zum ersten führen wir `session.run()` mit `train_step` aus, also man berechnet die Kostenfunktion und den Optimierer. Deswegen geben wir die Trainingsdaten ein und alle 10 Epochen wollen wir die Genauigkeit evaluiren.

Dafür verwenden wir `tf.argmax` um die Stelle der größten Zahl bei der Vorhersage und bei den echten Klassen zu vergleichen. Das gibt uns eine Liste von booleschen Werten (`True` oder `False`). Mithilfe von `cast(tf.float32)`, wandeln wir die boolesche Werte in `1` oder `0` um. Schließlich berechnen wir den Mittelwert dieser Liste. So bekommen wir eine Genauigkeit als Prozent. 

In [14]:
# Trainingschleife 
for epoch in range(300):
    for i in range(train_x.shape[0]):
        sess.run(train_step, feed_dict={X: train_x[i:i+1], Y: train_y[i: i+1]})                                                 
        
    if (epoch % 10 == 0):
        correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(Y, 1))                                                      
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))                                                       
        print("Epoch %d: Accuraccy = %f, Loss = %f" % (                                                                          
                          epoch,                                                                                                   
                          sess.run(accuracy, feed_dict={X: test_x, Y: test_y}),                                                    
                          sess.run(loss, feed_dict={X: train_x, Y: train_y})))
    

Epoch 0: Accuraccy = 0.263158, Loss = 1.199225
Epoch 10: Accuraccy = 0.263158, Loss = 1.194546
Epoch 20: Accuraccy = 0.263158, Loss = 1.192756
Epoch 30: Accuraccy = 0.631579, Loss = 0.879430
Epoch 40: Accuraccy = 0.631579, Loss = 0.868170
Epoch 50: Accuraccy = 0.631579, Loss = 0.860208
Epoch 60: Accuraccy = 0.973684, Loss = 0.741950
Epoch 70: Accuraccy = 1.000000, Loss = 0.658429
Epoch 80: Accuraccy = 1.000000, Loss = 0.637557
Epoch 90: Accuraccy = 1.000000, Loss = 0.628275
Epoch 100: Accuraccy = 1.000000, Loss = 0.622634
Epoch 110: Accuraccy = 1.000000, Loss = 0.618595
Epoch 120: Accuraccy = 1.000000, Loss = 0.615438
Epoch 130: Accuraccy = 1.000000, Loss = 0.612846
Epoch 140: Accuraccy = 1.000000, Loss = 0.610648
Epoch 150: Accuraccy = 1.000000, Loss = 0.608746
Epoch 160: Accuraccy = 1.000000, Loss = 0.607075
Epoch 170: Accuraccy = 1.000000, Loss = 0.605590
Epoch 180: Accuraccy = 1.000000, Loss = 0.604258
Epoch 190: Accuraccy = 1.000000, Loss = 0.603055
Epoch 200: Accuraccy = 1.000000

Man sieht, dass die Genauigkeit steigt und die Kostenfunktion sinkt. Man kann auch die Lernrate und die Anzahl von Neuronen in einer Schicht anpassen um die Veränderungen in der Genaigkeit sich anzuschauen.

### Was kann man verbessern?

- Man kann immer den Anzahl von Schichten und Neuronen anpassen
- die Lernrate kann auch angepasst werden
- um bessere Modelle zu bekommen, soll man die Daten bei jeder Epoche schlurfen
- für bessere Laufzeit, soll man die Stichproben nicht eine nach den anderen dem neuronalen Netzes eingeben, sondern die in Stapel zusammenfassen. Die größe der Stapel hängt von der Anzahl von Merkmalen und dem vorhandenen Haupt- oder Videospeicherplatz ab.

### Ressoursen

- Git Repository mit dem kompletten Code - [Link](https://github.com/emomicrowave/machine-learning-tutorials/tree/master/91_neural_networks)
- Wie wählt man eine optimale Anzahl von Zwischenschichten und deren Nuronen? (Enghlisch) - [Link](https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw)